In [1]:
import numpy as np
import math
from qiskit import Aer, IBMQ, execute
from qiskit.quantum_info.states.measures import state_fidelity
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.extensions.standard import HGate, SGate, SdgGate, XGate
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

def reverseBits(num,bitSize): 
    binary = bin(num)
    reverse = binary[-1:1:-1] 
    reverse = reverse + (bitSize - len(reverse))*'0'
    return int(reverse,2)

def simulate_circ(circ):
    backend = Aer.get_backend('statevector_simulator')
    job = execute(circ, backend=backend)
    result = job.result()
    outputstate = result.get_statevector(circ)
    outputstate_ordered = [0 for sv in outputstate]
    for i, sv in enumerate(outputstate):
        reverse_i = reverseBits(i,len(circ.qubits))
        outputstate_ordered[reverse_i] = sv
    output_prob = [np.power(np.absolute(x),2) for x in outputstate_ordered]
    # To use probabilities, use this return
    # return output_prob
    # To use state vectors, use this return
    return outputstate_ordered

def cross_entropy(d1,d2):
    h = 0
    for p,q in zip(d1,d2):
        if p==0:
            h += 0
        else:
            h+= -p*np.log(q)
    return h

q = QuantumRegister(3,'q')
circ = QuantumCircuit(q)
circ.h(q)
circ.t(q)
circ.cz(q[0],q[1])
circ.ry(math.pi/2,q[0])
circ.cz(q[1],q[2])
circ.t(q[0])
circ.rx(math.pi/2,q[1])
circ.ry(math.pi/2,q[2])
circ.h(q[0])
circ.t(q[1:])
circ.h(q[1:])
full_circ_sim = simulate_circ(circ)
print('Original circuit')
print(circ)
print('Cut at: (q[1], 2)')

q = QuantumRegister(2,'q')
cluster_0 = QuantumCircuit(q)
cluster_0.h(q)
cluster_0.t(q)
cluster_0.cz(q[0],q[1])
cluster_0.ry(math.pi/2,q[0])
cluster_0.t(q[0])
cluster_0.h(q[0])
print('Cluster 0:')
print(cluster_0)

q = QuantumRegister(2,'q')
cluster_1 = QuantumCircuit(q)
cluster_1.h(q[1])
cluster_1.t(q[1])
cluster_1.cz(q[0],q[1])
cluster_1.rx(math.pi/2,q[0])
cluster_1.ry(math.pi/2,q[1])
cluster_1.t(q)
cluster_1.h(q)
print('Cluster 1:')
print(cluster_1)
clusters = [cluster_0,cluster_1]

Original circuit
        ┌───┐┌───┐   ┌────────────┐    ┌───┐     ┌───┐     
q_0: |0>┤ H ├┤ T ├─■─┤ Ry(1.5708) ├────┤ T ├─────┤ H ├─────
        ├───┤├───┤ │ └────────────┘┌───┴───┴────┐├───┤┌───┐
q_1: |0>┤ H ├┤ T ├─■───────■───────┤ Rx(1.5708) ├┤ T ├┤ H ├
        ├───┤├───┤         │       ├────────────┤├───┤├───┤
q_2: |0>┤ H ├┤ T ├─────────■───────┤ Ry(1.5708) ├┤ T ├┤ H ├
        └───┘└───┘                 └────────────┘└───┘└───┘
Cut at: (q[1], 2)
Cluster 0:
        ┌───┐┌───┐   ┌────────────┐┌───┐┌───┐
q_0: |0>┤ H ├┤ T ├─■─┤ Ry(1.5708) ├┤ T ├┤ H ├
        ├───┤├───┤ │ └────────────┘└───┘└───┘
q_1: |0>┤ H ├┤ T ├─■─────────────────────────
        └───┘└───┘                           
Cluster 1:
                     ┌────────────┐┌───┐┌───┐
q_0: |0>───────────■─┤ Rx(1.5708) ├┤ T ├┤ H ├
        ┌───┐┌───┐ │ ├────────────┤├───┤├───┤
q_1: |0>┤ H ├┤ T ├─■─┤ Ry(1.5708) ├┤ T ├┤ H ├
        └───┘└───┘   └────────────┘└───┘└───┘


In [2]:
print('*'*20,'Manual Construction','*'*20)

print('S = 1')
cluster_0_circ = clusters[0]
cluster_0_dag = circuit_to_dag(cluster_0_circ)
cluster_0_circ = dag_to_circuit(cluster_0_dag)
cluster_0 = simulate_circ(cluster_0_circ)
cluster_0_collapsed = [cluster_0[0]+cluster_0[1],cluster_0[2]+cluster_0[3]]
cluster_1_circ = clusters[1]
cluster_1_dag = circuit_to_dag(cluster_1_circ)
cluster_1_circ = dag_to_circuit(cluster_1_dag)
cluster_1 = simulate_circ(cluster_1_circ)
term_1 = np.kron(cluster_0_collapsed,cluster_1)*(1/2)
print('Cluster 0 Evaluate (''zero'', ''zero'') (''I'', ''I'')')
print(cluster_0_circ)
print(cluster_0_collapsed)
print('Cluster 1 Evaluate (''zero'', ''zero'') (''I'', ''I'')')
print(cluster_1_circ)
print(cluster_1)
print('-'*100)

print('S = 2')
cluster_0_circ = clusters[0]
cluster_0_dag = circuit_to_dag(cluster_0_circ)
cluster_0_circ = dag_to_circuit(cluster_0_dag)
cluster_0 = simulate_circ(cluster_0_circ)
cluster_0_collapsed = [cluster_0[0]+cluster_0[1],cluster_0[2]+cluster_0[3]]
cluster_1_circ = clusters[1]
cluster_1_dag = circuit_to_dag(cluster_1_circ)
cluster_1_dag.apply_operation_front(op=XGate(),qargs=[cluster_1_circ.qubits[0]],cargs=[])
cluster_1_circ = dag_to_circuit(cluster_1_dag)
cluster_1 = simulate_circ(cluster_1_circ)
term_2 = np.kron(cluster_0_collapsed,cluster_1)*(1/2)
print('Cluster 0 Evaluate (''zero'', ''zero'') (''I'', ''I'')')
print(cluster_0_circ)
print(cluster_0_collapsed)
print('Cluster 1 Evaluate (''one'', ''zero'') (''I'', ''I'')')
print(cluster_1_circ)
print(cluster_1)
print('-'*100)

print('S = 3')
cluster_0_circ = clusters[0]
cluster_0_dag = circuit_to_dag(cluster_0_circ)
cluster_0_dag.apply_operation_back(op=HGate(),qargs=[cluster_0_circ.qubits[1]],cargs=[])
cluster_0_circ = dag_to_circuit(cluster_0_dag)
cluster_0 = simulate_circ(cluster_0_circ)
cluster_0_collapsed = [cluster_0[0]-cluster_0[1],cluster_0[2]-cluster_0[3]]
cluster_1_circ = clusters[1]
cluster_1_dag = circuit_to_dag(cluster_1_circ)
cluster_1_dag.apply_operation_front(op=HGate(),qargs=[cluster_1_circ.qubits[0]],cargs=[])
cluster_1_circ = dag_to_circuit(cluster_1_dag)
cluster_1 = simulate_circ(cluster_1_circ)
term_3 = np.kron(cluster_0_collapsed,cluster_1)*(1/2)
print('Cluster 0 Evaluate (''zero'', ''zero'') (''I'', ''X'')')
print(cluster_0_circ)
print(cluster_0_collapsed)
print('Cluster 1 Evaluate (''plus'', ''zero'') (''I'', ''I'')')
print(cluster_1_circ)
print(cluster_1)
print('-'*100)

print('S = 4')
cluster_0_circ = clusters[0]
cluster_0_dag = circuit_to_dag(cluster_0_circ)
cluster_0_dag.apply_operation_back(op=HGate(),qargs=[cluster_0_circ.qubits[1]],cargs=[])
cluster_0_circ = dag_to_circuit(cluster_0_dag)
cluster_0 = simulate_circ(cluster_0_circ)
cluster_0_collapsed = [cluster_0[0]-cluster_0[1],cluster_0[2]-cluster_0[3]]
cluster_1_circ = clusters[1]
cluster_1_dag = circuit_to_dag(cluster_1_circ)
cluster_1_dag.apply_operation_front(op=HGate(),qargs=[cluster_1_circ.qubits[0]],cargs=[])
cluster_1_dag.apply_operation_front(op=XGate(),qargs=[cluster_1_circ.qubits[0]],cargs=[])
cluster_1_circ = dag_to_circuit(cluster_1_dag)
cluster_1 = simulate_circ(cluster_1_circ)
term_4 = np.kron(cluster_0_collapsed,cluster_1)*(-1/2)
print('Cluster 0 Evaluate (''zero'', ''zero'') (''I'', ''X'')')
print(cluster_0_circ)
print(cluster_0_collapsed)
print('Cluster 1 Evaluate (''minus'', ''zero'') (''I'', ''I'')')
print(cluster_1_circ)
print(cluster_1)
print('-'*100)

print('S = 5')
cluster_0_circ = clusters[0]
cluster_0_dag = circuit_to_dag(cluster_0_circ)
cluster_0_dag.apply_operation_back(op=SdgGate(),qargs=[cluster_0_circ.qubits[1]],cargs=[])
cluster_0_dag.apply_operation_back(op=HGate(),qargs=[cluster_0_circ.qubits[1]],cargs=[])
cluster_0_circ = dag_to_circuit(cluster_0_dag)
cluster_0 = simulate_circ(cluster_0_circ)
cluster_0_collapsed = [cluster_0[0]-cluster_0[1],cluster_0[2]-cluster_0[3]]
cluster_1_circ = clusters[1]
cluster_1_dag = circuit_to_dag(cluster_1_circ)
cluster_1_dag.apply_operation_front(op=SGate(),qargs=[cluster_1_circ.qubits[0]],cargs=[])
cluster_1_dag.apply_operation_front(op=HGate(),qargs=[cluster_1_circ.qubits[0]],cargs=[])
cluster_1_circ = dag_to_circuit(cluster_1_dag)
cluster_1 = simulate_circ(cluster_1_circ)
term_5 = np.kron(cluster_0_collapsed,cluster_1)*(1/2)
print('Cluster 0 Evaluate (''zero'', ''zero'') (''I'', ''Y'')')
print(cluster_0_circ)
print(cluster_0_collapsed)
print('Cluster 1 Evaluate (''plus_i'', ''zero'') (''I'', ''I'')')
print(cluster_1_circ)
print(cluster_1)
print('-'*100)

print('S = 6')
cluster_0_circ = clusters[0]
cluster_0_dag = circuit_to_dag(cluster_0_circ)
cluster_0_dag.apply_operation_back(op=SdgGate(),qargs=[cluster_0_circ.qubits[1]],cargs=[])
cluster_0_dag.apply_operation_back(op=HGate(),qargs=[cluster_0_circ.qubits[1]],cargs=[])
cluster_0_circ = dag_to_circuit(cluster_0_dag)
cluster_0 = simulate_circ(cluster_0_circ)
cluster_0_collapsed = [cluster_0[0]-cluster_0[1],cluster_0[2]-cluster_0[3]]
cluster_1_circ = clusters[1]
cluster_1_dag = circuit_to_dag(cluster_1_circ)
cluster_1_dag.apply_operation_front(op=SGate(),qargs=[cluster_1_circ.qubits[0]],cargs=[])
cluster_1_dag.apply_operation_front(op=HGate(),qargs=[cluster_1_circ.qubits[0]],cargs=[])
cluster_1_dag.apply_operation_front(op=XGate(),qargs=[cluster_1_circ.qubits[0]],cargs=[])
cluster_1_circ = dag_to_circuit(cluster_1_dag)
cluster_1 = simulate_circ(cluster_1_circ)
term_6 = np.kron(cluster_0_collapsed,cluster_1)*(-1/2)
print('Cluster 0 Evaluate (''zero'', ''zero'') (''I'', ''Y'')')
print(cluster_0_circ)
print(cluster_0_collapsed)
print('Cluster 1 Evaluate (''minus_i'', ''zero'') (''I'', ''I'')')
print(cluster_1_circ)
print(cluster_1)
print('-'*100)

print('S = 7')
cluster_0_circ = clusters[0]
cluster_0_dag = circuit_to_dag(cluster_0_circ)
cluster_0_circ = dag_to_circuit(cluster_0_dag)
cluster_0 = simulate_circ(cluster_0_circ)
cluster_0_collapsed = [cluster_0[0]-cluster_0[1],cluster_0[2]-cluster_0[3]]
cluster_1_circ = clusters[1]
cluster_1_dag = circuit_to_dag(cluster_1_circ)
cluster_1_circ = dag_to_circuit(cluster_1_dag)
cluster_1 = simulate_circ(cluster_1_circ)
term_7 = np.kron(cluster_0_collapsed,cluster_1)*(1/2)
print('Cluster 0 Evaluate (''zero'', ''zero'') (''I'', ''Z'')')
print(cluster_0_circ)
print(cluster_0_collapsed)
print('Cluster 1 Evaluate (''zero'', ''zero'') (''I'', ''I'')')
print(cluster_1_circ)
print(cluster_1)
print('-'*100)

print('S = 8')
cluster_0_circ = clusters[0]
cluster_0_dag = circuit_to_dag(cluster_0_circ)
cluster_0_circ = dag_to_circuit(cluster_0_dag)
cluster_0 = simulate_circ(cluster_0_circ)
cluster_0_collapsed = [cluster_0[0]-cluster_0[1],cluster_0[2]-cluster_0[3]]
cluster_1_circ = clusters[1]
cluster_1_dag = circuit_to_dag(cluster_1_circ)
cluster_1_dag.apply_operation_front(op=XGate(),qargs=[cluster_1_circ.qubits[0]],cargs=[])
cluster_1_circ = dag_to_circuit(cluster_1_dag)
cluster_1 = simulate_circ(cluster_1_circ)
term_8 = np.kron(cluster_0_collapsed,cluster_1)*(-1/2)
print('Cluster 0 Evaluate (''zero'', ''zero'') (''I'', ''Z'')')
print(cluster_0_circ)
print(cluster_0_collapsed)
print('Cluster 1 Evaluate (''one'', ''zero'') (''I'', ''I'')')
print(cluster_1_circ)
print(cluster_1)
print('-'*100)

manual_reconstruction = term_1+term_2+term_3+term_4+term_5+term_6+term_7+term_8
summation = sum(manual_reconstruction)
if isinstance(summation, complex):
    print('manual reconstruction fidelity = ',state_fidelity(manual_reconstruction,full_circ_sim))
    print('identical distributions fidelity = ',state_fidelity(full_circ_sim,full_circ_sim))
else:
    print('sum of prob =',summation)
    print('manual reconstruction cross entropy = ',cross_entropy(manual_reconstruction,full_circ_sim))
    print('identical distributions cross entropy = ',cross_entropy(full_circ_sim,full_circ_sim))

******************** Manual Construction ********************
S = 1
Cluster 0 Evaluate (zero, zero) (I, I)
        ┌───┐┌───┐   ┌────────────┐┌───┐┌───┐
q_0: |0>┤ H ├┤ T ├─■─┤ Ry(1.5708) ├┤ T ├┤ H ├
        ├───┤├───┤ │ └────────────┘└───┘└───┘
q_1: |0>┤ H ├┤ T ├─■─────────────────────────
        └───┘└───┘                           
[(0.6035533905932738+0.7499999999999998j), (-0.10355339059327372-0.2500000000000001j)]
Cluster 1 Evaluate (zero, zero) (I, I)
                     ┌────────────┐┌───┐┌───┐
q_0: |0>───────────■─┤ Rx(1.5708) ├┤ T ├┤ H ├
        ┌───┐┌───┐ │ ├────────────┤├───┤├───┤
q_1: |0>┤ H ├┤ T ├─■─┤ Ry(1.5708) ├┤ T ├┤ H ├
        └───┘└───┘   └────────────┘└───┘└───┘
[(0.42677669529663687+0.1767766952966367j), (-0.4267766952966368-0.6767766952966368j), (-0.07322330470336302+0.17677669529663695j), (0.28033008588991065-0.1767766952966369j)]
----------------------------------------------------------------------------------------------------
S = 2
Cluster 0 Evaluate (zero,